# Convolutional Neural Networks

## Load Data

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import KFold

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [4]:
images_train = np.expand_dims(np.load("../data/images/images_train.npy") / 255.0, -1)
images_test = np.expand_dims(np.load("../data/images/images_test.npy") / 255.0, -1)
num_samples_train = images_train.shape[0]
num_samples_test = images_test.shape[0]

hog_train = np.load("../data/hog/hog_train.npy").reshape(num_samples_train, -1) * 255 / 255.0
hog_test = np.load("../data/hog/hog_test.npy").reshape(num_samples_test, -1) * 255 / 255.0

y_train = np.load("../data/images/label_ids_train.npy")
y_test = np.load("../data/images/label_ids_test.npy")


In [5]:
labels_train = np.load("../data/images/labels_train.npy")
class_lookup = list(set(zip(y_train, labels_train)))
class_lookup = {id_value: class_value for id_value, class_value in class_lookup}
class_lookup


{8: 'impacted',
 5: 'pathological',
 9: 'avulsion',
 4: 'dislocation',
 1: 'spiral',
 2: 'greenstick',
 3: 'comminuted',
 7: 'oblique',
 6: 'longitudinal',
 0: 'hairline'}

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
for train, test in kfold.split(images_train, y_train):

    image_input = layers.Input(shape=(images_train.shape[1], images_train.shape[2], images_train.shape[3]))
    x = layers.Conv2D(16, (3, 3), activation='relu')(image_input)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(32, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(32, (3, 3), activation='relu')(x)
    x = layers.Flatten()(x)
    
    hog_input = layers.Input(shape=(hog_train.shape[1],))
    x = layers.Concatenate()([x, hog_input])
    
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(10, activation='softmax')(x)

    model = models.Model(inputs=[image_input, hog_input], outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    # Fit data to model
    history = model.fit(images_train[train], y_train[train],
              batch_size=32,
              epochs=10,
              verbose=1)
    
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    # Increase fold number
    fold_no = fold_no + 1


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
 5/25 ━━━━━━━━━━━━━━━━━━━━ 2:48 8s/step - accuracy: 0.1388 - loss: 13.5314

In [6]:
y_pred = model.predict([images_test, hog_test])
y_pred = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report).transpose()
report.index = report.index.map(lambda x: class_lookup[int(x)] if x.isdigit() else x)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
report


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step
Accuracy: 0.2571428571428571
Classification Report:


/home/davidsolow/.pyenv/versions/3.11.1/envs/fracture-classification/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/davidsolow/.pyenv/versions/3.11.1/envs/fracture-classification/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/davidsolow/.pyenv/versions/3.11.1/envs/fracture-classification/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use 

,precision,recall,f1-score,support
hairline,0.125000,0.100000,0.111111,10.000000
spiral,0.333333,0.166667,0.222222,12.000000
greenstick,0.368421,0.437500,0.400000,16.000000
comminuted,0.235294,0.285714,0.258065,14.000000
dislocation,0.263158,0.526316,0.350877,19.000000
pathological,0.200000,0.277778,0.232558,18.000000
longitudinal,0.250000,0.250000,0.250000,12.000000
oblique,0.363636,0.250000,0.296296,16.000000
impacted,0.000000,0.000000,0.000000,9.000000
avulsion,0.000000,0.000000,0.000000,14.000000
